In [77]:
import qiskit
import numpy as np
import qiskit.quantum_info as qi
from qiskit.quantum_info.operators import Operator

In [78]:
I = [[1,0],[0, 1]]
Z = qi.Pauli.pauli_single(1, 0, "Z").to_matrix()
Y = qi.Pauli.pauli_single(1, 0, "Y").to_matrix()
X = qi.Pauli.pauli_single(1, 0, "X").to_matrix()

In [87]:
# construct magic state on single qubit
t0 = 1/2 * qi.DensityMatrix(I + 1/np.sqrt(3)*(X + Y + Z))
t1 = 1/2 * qi.DensityMatrix(I - 1/np.sqrt(3)*(X + Y + Z))
eps = .2
rho_t = (1-eps) * t0 + eps * t1

# tensor product to get the full input state
state = rho_t.copy()
N = 5 # number of qubits
for i in range(N-1):
    state = state.tensor(rho_t)

In [88]:
# construct circuit
circuit = qiskit.QuantumCircuit(N)
# layer 1
circuit.cz(3, 0)
circuit.cz(3, 2)
circuit.cx(3, 4)
circuit.cz(3, 4)



# layer 2
circuit.cz(2, 0)
circuit.cz(2, 1)
circuit.cx(2, 4)
# layer 3
circuit.cx(1, 4)
# layer 4
circuit.cx(0, 4)
circuit.cz(0, 4)


# layer 5
circuit.z(0)
circuit.z(3)
circuit.z(4)

circuit.h(0)
# circuit.x(1)
circuit.h(1)
circuit.h(2)
circuit.h(3)
circuit.h(4)

circuit.y(4)

In [89]:
print(circuit)

                                               ┌───┐┌───┐     
q_0: ─■──────■───────────────────────────■───■─┤ Z ├┤ H ├─────
      │      │                    ┌───┐  │   │ └───┘└───┘     
q_1: ─┼──────┼───■─────────────■──┤ H ├──┼───┼────────────────
      │      │   │      ┌───┐  │  └───┘  │   │                
q_2: ─┼──■───■───■───■──┤ H ├──┼─────────┼───┼────────────────
      │  │           │  ├───┤  │  ┌───┐  │   │                
q_3: ─■──■───■───■───┼──┤ Z ├──┼──┤ H ├──┼───┼────────────────
           ┌─┴─┐ │ ┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐ │ ┌───┐┌───┐┌───┐
q_4: ──────┤ X ├─■─┤ X ├─────┤ X ├─────┤ X ├─■─┤ Z ├┤ H ├┤ Y ├
           └───┘   └───┘     └───┘     └───┘   └───┘└───┘└───┘


In [90]:

final_state = state.evolve(circuit)
# check state of rdm on qubit 5

# measure
# check state of rdm on qubit 5
while True:
    string, dm = final_state.measure(range(4)) # measure first four bits
    if string == '0000':
        break
# print(string)
# string, dm = final_state.measure(range(4)) # measure first four bits


rho_t_new = qi.partial_trace(dm, range(4))
def magic_fidelity(dm):
    return max(qi.state_fidelity(t0, dm), qi.state_fidelity(t1, dm))

print(magic_fidelity(rho_t))
print(magic_fidelity(rho_t_new))
print("Initial T fidelity: ", qi.state_fidelity(t0, rho_t))
print("Final T fidelity: ", qi.state_fidelity(t0, rho_t_new))

0.8000000066804441
0.7747368490822446
Initial T fidelity:  0.8000000066804441
Final T fidelity:  0.7747368490822446
